# 文本情感分析

## 📋 概述

文本情感分析是股票分析中的重要方法，通过分析文本数据（新闻、社交媒体、研报等）的情感倾向来预测市场情绪和股价走势。本章节将详细介绍文本情感分析的方法，包括情感词典、机器学习方法、深度学习方法等。

**学习方式**：本文件是Jupyter Notebook格式，你可以边看边运行代码，通过实际代码示例理解文本情感分析。


In [ ]:
# 安装必要的库（如果还没有安装）
# !pip install pandas numpy matplotlib jieba scikit-learn

# 导入库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

# 尝试导入jieba（中文分词）
try:
    import jieba
    JIEBA_AVAILABLE = True
except ImportError:
    JIEBA_AVAILABLE = False
    print("jieba不可用，将使用简化版本")

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

print("环境准备完成！")


## 📖 核心内容

### 1. 文本情感分析的基本概念

#### 1.1 什么是情感分析

**情感分析（Sentiment Analysis）**：
- 分析文本的情感倾向
- 判断文本是正面、负面还是中性
- 量化情感强度

#### 1.2 应用场景

**市场情绪分析**：
- 分析新闻情感
- 分析社交媒体情感
- 分析研报情感

**预测应用**：
- 情感与股价的关系
- 情感指标构建
- 交易信号生成

让我们创建一个简单的情感分析工具：


In [ ]:
# 简单的情感分析工具
class SentimentAnalyzer:
    """情感分析器"""
    
    def __init__(self):
        # 正面词汇
        self.positive_words = ['涨', '上涨', '利好', '看好', '推荐', '买入', '强势', 
                              '突破', '增长', '盈利', '优秀', '良好', '积极', '乐观']
        # 负面词汇
        self.negative_words = ['跌', '下跌', '利空', '看空', '卖出', '弱势', 
                              '破位', '下降', '亏损', '糟糕', '不良', '消极', '悲观']
        # 程度词
        self.intensity_words = {
            '非常': 1.5, '极其': 1.5, '特别': 1.3, '很': 1.2, '较': 1.1,
            '稍微': 0.8, '略微': 0.8, '有点': 0.9
        }
    
    def analyze(self, text):
        """
        分析文本情感
        
        参数:
        text: 文本内容
        
        返回:
        sentiment_score: 情感分数（-1到1之间）
        """
        if not text or pd.isna(text):
            return 0
        
        text = str(text)
        score = 0
        
        # 检查正面词汇
        for word in self.positive_words:
            count = text.count(word)
            if count > 0:
                score += count * 0.1
        
        # 检查负面词汇
        for word in self.negative_words:
            count = text.count(word)
            if count > 0:
                score -= count * 0.1
        
        # 调整程度
        for intensity_word, multiplier in self.intensity_words.items():
            if intensity_word in text:
                # 简化处理：如果程度词在文本中，调整分数
                score *= multiplier
        
        # 归一化到-1到1之间
        score = np.clip(score, -1, 1)
        
        return score
    
    def classify(self, text):
        """分类情感"""
        score = self.analyze(text)
        if score > 0.1:
            return '正面'
        elif score < -0.1:
            return '负面'
        else:
            return '中性'

# 示例文本
sample_texts = [
    "该股票今日大幅上涨，市场看好其未来发展前景",
    "公司业绩下滑，股价可能继续下跌",
    "市场表现平稳，投资者保持观望态度",
    "非常看好这只股票，强烈推荐买入",
    "利空消息不断，建议卖出"
]

# 创建情感分析器
analyzer = SentimentAnalyzer()

# 分析情感
results = []
for text in sample_texts:
    score = analyzer.analyze(text)
    sentiment = analyzer.classify(text)
    results.append({
        '文本': text,
        '情感分数': score,
        '情感分类': sentiment
    })

results_df = pd.DataFrame(results)
print("文本情感分析结果：")
print(results_df.to_string(index=False))


In [ ]:
# 可视化情感分析结果
fig, ax = plt.subplots(figsize=(12, 6))

scores = results_df['情感分数'].values
colors = ['green' if s > 0.1 else 'red' if s < -0.1 else 'gray' for s in scores]

bars = ax.barh(range(len(scores)), scores, color=colors, alpha=0.7, edgecolor='black')
ax.axvline(x=0, color='black', linestyle='-', linewidth=1)
ax.set_yticks(range(len(sample_texts)))
ax.set_yticklabels([f"文本{i+1}" for i in range(len(sample_texts))])
ax.set_xlabel('情感分数')
ax.set_title('文本情感分析结果', fontsize=14, fontweight='bold')
ax.set_xlim(-1, 1)
ax.grid(True, alpha=0.3, axis='x')

# 标注数值
for i, (score, sentiment) in enumerate(zip(scores, results_df['情感分类'])):
    ax.text(score, i, f'{score:.2f} ({sentiment})', 
           ha='left' if score >= 0 else 'right', va='center', fontweight='bold')

plt.tight_layout()
plt.show()


### 3. 情感分析与股价关系

让我们分析情感分数与股价的关系：


In [ ]:
# 模拟情感数据与股价关系
np.random.seed(42)
n_days = 100
dates = pd.date_range(start='2024-01-01', periods=n_days, freq='D')
dates = dates[dates.weekday < 5]

# 生成情感分数（与未来收益相关）
sentiment_scores = np.random.normal(0, 0.3, len(dates))
# 情感分数影响未来收益
future_returns = sentiment_scores * 0.02 + np.random.randn(len(dates)) * 0.01

# 计算累计价格
prices = 10 * (1 + future_returns).cumprod()

# 可视化
fig, axes = plt.subplots(2, 1, figsize=(15, 10), sharex=True)

axes[0].plot(dates, sentiment_scores, linewidth=1.5, color='blue', alpha=0.7, label='情感分数')
axes[0].axhline(y=0, color='black', linestyle='-', linewidth=0.5)
axes[0].fill_between(dates, 0, sentiment_scores, where=(sentiment_scores > 0), 
                     alpha=0.3, color='green', label='正面情感')
axes[0].fill_between(dates, 0, sentiment_scores, where=(sentiment_scores < 0), 
                     alpha=0.3, color='red', label='负面情感')
axes[0].set_title('情感分数时间序列', fontsize=14, fontweight='bold')
axes[0].set_ylabel('情感分数')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(dates, prices, linewidth=2, color='black', alpha=0.7, label='股价')
axes[1].set_title('股价走势', fontsize=14, fontweight='bold')
axes[1].set_xlabel('日期')
axes[1].set_ylabel('价格')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# 计算相关性
correlation = np.corrcoef(sentiment_scores, future_returns)[0, 1]
print(f"\n情感分数与未来收益率的相关性：{correlation:.4f}")
